# Recommendations: Part 2

In the 2nd part of our recommendations exercise, you will use the PageRank algorithm to make article recommendations to an author. 
Execute the code to import the libraries (remember to unset Reset all runtimes before running):

In [1]:
from py2neo import Graph
import pandas as pd

import matplotlib 
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_colwidth', 100)

Update the cell below to use the IP Address, Bolt Port, and Password, as you did previously.

In [2]:
# Change the line of code below to use the Bolt URL,  and Password of your Neo4j database instance.
# graph = Graph("<Bolt URL>", auth=("neo4j", "<Password>")) 
 
graph = Graph("bolt://localhost:7687", auth=("neo4j", "letmein"))

## PageRank

PageRank is an algorithm that measures the transitive influence or connectivity of nodes. It can be computed by either iteratively distributing one node’s rank (originally based on degree) over its neighbors or by randomly traversing the graph and counting the frequency of hitting each node during these walks.

Run this PageRank code over the whole graph to find out the most influential article in terms of citations:

In [4]:
query = """
CALL gds.pageRank.write({
  nodeProjection:'Article', 
  relationshipProjection:'CITED',
  writeProperty:'pagerank'})
"""
graph.run(query).data()

[{'centralityDistribution': {'max': 93.94335842132568,
   'mean': 0.22228103068016428,
   'min': 0.14999961853027344,
   'p1': 0.14999961853027344,
   'p10': 0.14999961853027344,
   'p100': 93.94335842132568,
   'p25': 0.14999961853027344,
   'p5': 0.14999961853027344,
   'p50': 0.14999961853027344,
   'p75': 0.14999961853027344,
   'p90': 0.2774991989135742,
   'p95': 0.43799686431884766,
   'p99': 1.3121633529663086,
   'stdDev': 0.6853762857474536},
  'computeMillis': 545,
  'configuration': {'cacheWeights': False,
   'concurrency': 4,
   'dampingFactor': 0.85,
   'maxIterations': 20,
   'nodeLabels': ['*'],
   'relationshipTypes': ['*'],
   'relationshipWeightProperty': None,
   'sourceNodes': [],
   'sudo': False,
   'tolerance': 1e-07,
   'writeConcurrency': 4,
   'writeProperty': 'pagerank'},
  'createMillis': 204,
  'didConverge': False,
  'nodePropertiesWritten': 51956,
  'postProcessingMillis': 84,
  'ranIterations': 20,
  'writeMillis': 897}]

This query stores a 'pagerank' property on each node. Execute this code to view the most influential articles:

In [5]:
query = """
MATCH (a:Article)
RETURN a.title as article,
       a.pagerank as score
ORDER BY score DESC 
LIMIT 10
"""
graph.run(query).to_data_frame()

,article,score
0,A method for obtaining digital signatures and public-key cryptosystems,93.943
1,Secure communications over insecure channels,79.869
2,Rough sets,25.609
3,An axiomatic basis for computer programming,23.029
4,"Pastry: Scalable, Decentralized Object Location, and Routing for Large-Scale Peer-to-Peer Systems",21.470
5,SCRIBE: The Design of a Large-Scale Event Notification Infrastructure,19.486
6,A field study of the software design process for large systems,19.028
7,Productivity factors and programming environments,18.499
8,Analyzing medium-scale software development,16.453
9,A Calculus of Communicating Systems,15.431


## Personalized PageRank

Personalized PageRank is a variant of PageRank that allows us to find influential nodes based on a set of source nodes.

For example, rather than finding the overall most influential articles, you could instead, find the most influential articles with respect to a given author.
Execute this code to use a personalized PageRank algorithm:

In [6]:
query = """
MATCH (a:Author {name: $author})<-[:AUTHOR]-(article)-[:CITED]->(other)
WITH collect(article) + collect(other) AS sourceNodes
CALL gds.pageRank.stream({
  nodeProjection:'Article',
  relationshipProjection:'CITED',
  sourceNodes: sourceNodes})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).title AS article, score
ORDER BY score DESC
LIMIT 10
"""

author_name = "Peter G. Neumann"
graph.run(query, {"author": author_name}).to_data_frame()

,article,score
0,A technique for software module specification with examples,0.359
1,A messy state of the union: taming the composite state machines of TLS,0.332
2,Risks of automation: a cautionary total-system perspective of our cyberfuture,0.278
3,Risks of e-voting,0.278
4,The foresight saga,0.278
5,Public interest and the NII,0.278
6,Crypto policy perspectives,0.278
7,Password security: a case history,0.278
8,The challenges of partially automated driving,0.268
9,Proof techniques for hierarchically structured programs,0.248


## Topic Sensitive Search

You can also use Personalized PageRank to do 'Topic Specific PageRank'. 

When an author is searching for articles to read, they want that search to take themselves as authors into account. Two authors using the same search term would expect to see different results depending on their area of research.

Create a full text search index on the 'title' and 'abstract' properties of all nodes that have the label 'Article' by executing this code:

In [7]:
query = """
    CALL db.index.fulltext.createNodeIndex('articles', ['Article'], ['title', 'abstract'])
"""
graph.run(query).data()

[]

Check that the full text index has been created by running the following query:

In [10]:
query = """
CALL db.indexes()
YIELD name, state, populationPercent, uniqueness, type, entityType, labelsOrTypes, properties, provider
WHERE type = "FULLTEXT"
RETURN *
"""
graph.run(query).to_data_frame()

,entityType,labelsOrTypes,name,populationPercent,properties,provider,state,type,uniqueness
0,NODE,[Article],articles,100.000,"[title, abstract]",fulltext-1.0,ONLINE,FULLTEXT,NONUNIQUE


You can search the full text index like this:

In [12]:
query = """
CALL db.index.fulltext.queryNodes("articles", "open source")
YIELD node, score
RETURN node.title as article, score, [(author)<-[:AUTHOR]-(node) | author.name] AS authors
LIMIT 10
"""
graph.run(query).to_data_frame()

,article,authors,score
0,Progressive open source,"[Jamie Dinkelacker, Pankaj K. Garg, Dean Nelson, Rob Miller]",11.740
1,Open source application spaces: the 5th workshop on open source software engineering,"[Joseph Feller, Scott A. Hissam, Brian Fitzgerald, Krishna K Lakhani, Walt Scacchi]",11.150
2,Reusing Open-Source Software and Practices: The Impact of Open-Source on Commercial Vendors,"[Alan W. Brown, Grady Booch]",11.016
3,From Research Software to Open Source,[Susan L. Graham],10.328
4,The comment density of open source software code,"[Oliver Arafati, Dirk Riehle]",10.292
5,Software architecture in an open source world,[Roy T. Fielding],10.250
6,Managing a corporate open source software asset,"[James D. Herbsleb, Vijay K. Gurbani, Anita Garvert]",10.187
7,Organizational adoption of open source software: barriers and remedies,"[Anol Bhattacherjee, Areej M. Yassin, Del Nagy]",9.894
8,IBM's pragmatic embrace of open source,[Pamela Samuelson],9.849
9,Analysing the reliability of Open Source software projects,"[Lerina Aversano, Maria Tortorella]",9.683


Here is a query to find the authors that have published the most articles on 'open source':

In [13]:
query = """
CALL db.index.fulltext.queryNodes("articles", "open source")
YIELD node, score
MATCH (node)-[:AUTHOR]->(author)
RETURN author.name as author, sum(score) AS totalScore, collect(node.title) AS articles
ORDER By totalScore DESC
LIMIT 20
"""

graph.run(query).to_data_frame()

,articles,author,totalScore
0,"[Machine learning-based detection of open source license exceptions, Recommending source code fo...",Denys Poshyvanyk,51.326
1,"[Open source application spaces: the 5th workshop on open source software engineering, The 3rd w...",Brian Fitzgerald,49.406
2,"[Open source application spaces: the 5th workshop on open source software engineering, The 3rd w...",Joseph Feller,45.596
3,"[Managing a corporate open source software asset, A case study of a corporate open source develo...",James D. Herbsleb,38.699
4,"[Who should fix this bug, Hipikat: recommending pertinent software development artifacts, FEAT a...",Gail C. Murphy,36.538
5,"[Open source application spaces: the 5th workshop on open source software engineering, Experienc...",Walt Scacchi,34.336
6,"[A study of the quality-impacting practices of modern code review at Sony mobile, An empirical s...",Ahmed E. Hassan,33.800
7,"[Machine learning-based detection of open source license exceptions, Open source-style collabora...",Daniel M. German,33.478
8,"[Disseminating architectural knowledge on open-source projects: a case study of the book ""archit...",Martin P. Robillard,33.017
9,"[Open source application spaces: the 5th workshop on open source software engineering, The 3rd w...",Scott A. Hissam,32.555


Next, use full text search and Personalized PageRank to find interesting articles for different authors:

In [17]:
query = """
MATCH (a:Author {name: $author})<-[:AUTHOR]-(article)-[:CITED]->(other)
WITH a, collect(article) + collect(other) AS sourceNodes
CALL gds.pageRank.stream({
  nodeQuery: 'CALL db.index.fulltext.queryNodes("articles", $searchTerm)
   YIELD node, score
   RETURN id(node) as id',
  relationshipQuery: 'MATCH (a1:Article)-[:CITED]->(a2:Article) 
   RETURN id(a1) as source,id(a2) as target', 
  sourceNodes: sourceNodes,
  validateRelationships:false,
  parameters: {searchTerm: $searchTerm}})
YIELD nodeId, score
WITH gds.util.asNode(nodeId) AS n, score
WHERE not(exists((a)<-[:AUTHOR]-(n))) AND score > 0
RETURN n.title as article, score, [(n)-[:AUTHOR]->(author) | author.name][..5] AS authors
order by score desc limit 10
"""

params = {"author": "Tao Xie", "searchTerm": "open source"}
graph.run(query, params).to_data_frame()

,article,authors,score
0,Static detection of cross-site scripting vulnerabilities,"[Zhendong Su, Gary Wassermann]",0.386
1,Who should fix this bug,"[Gail C. Murphy, John Anvik, Lyndon Hiew]",0.278
2,"Automated, contract-based user testing of commercial-off-the-shelf components","[Yvan Labiche, Michal M. Sówka, Lionel C. Briand]",0.278
3,Concern graphs: finding and describing concerns using structural program dependencies,"[Martin P. Robillard, Gail C. Murphy]",0.278
4,Characterizing logging practices in open-source software,"[Yuanyuan Zhou, Ding Yuan, Soyeon Park]",0.278
5,Conceptual module querying for software reengineering,"[Elisa L. A. Baniassad, Gail C. Murphy]",0.236
6,Bandera: extracting finite-state models from Java source code,"[James C. Corbett, Shawn Laubach, John Hatcliff, Robby, Matthew B. Dwyer]",0.150
7,AsDroid: detecting stealthy behaviors in Android applications by user interface and program beha...,"[Xiangyu Zhang, Bin Liang, Jianjun Huang, Lin Tan, Peng Wang]",0.150
8,Semantics-based code search,[Steven P. Reiss],0.150
9,EXSYST: search-based GUI testing,"[Gordon Fraser, Florian Gross, Andreas Zeller]",0.128


Execute the same query with a different author:

In [18]:
params = {"author": "Marco Aurélio Gerosa", "searchTerm": "open source"}
graph.run(query, params).to_data_frame()

,article,authors,score
0,Toward an understanding of the motivation of open source software developers,"[Yunwen Ye, Kouichi Kishida]",0.388
1,Hipikat: recommending pertinent software development artifacts,"[Gail C. Murphy, Davor Cubranic]",0.322
2,Version Sensitive Editing: Change History as a Programming Tool,[David L. Atkins],0.274
3,Which bug should I fix: helping new developers onboard a new project,"[Jianguo Wang, Anita Sarma]",0.239
4,Tesseract: Interactive visual exploration of socio-technical relationships in software development,"[Patrick Wagstrom, Larry Maccherone, James D. Herbsleb, Anita Sarma]",0.203
5,Role Migration and Advancement Processes in OSSD Projects: A Comparative Case Study,"[Chris Jensen, Walt Scacchi]",0.175
6,Does the initial environment impact the future of developers,"[Audris Mockus, Minghui Zhou]",0.175
7,Unifying artifacts and activities in a visual tool for distributed software development teams,"[Jon Froehlich, Paul Dourish]",0.173
8,A case study of open source software development: the Apache server,"[Audris Mockus, James D. Herbsleb, Roy Fielding]",0.110
9,A case study of the evolution of Jun: an object-oriented open-source 3D multimedia library,"[A. Takasbima, Kouichi Kishida, Yoshiyuki Nishinaka, Kaoru Hayashi, Atsushi Aoki]",0.110
